# Task 3: A/B Hypothesis Testing


Imports

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt


Load Dataset

In [4]:
df = pd.read_csv("../data/insurance.txt", delimiter="|")
df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'], errors='coerce')
df['LossRatio'] = df['TotalClaims'] / df['TotalPremium']
df['LossRatio'] = df['LossRatio'].replace([np.inf, -np.inf], np.nan)
df['ClaimOccurred'] = df['TotalClaims'] > 0
df.head()


/tmp/ipykernel_171076/2342708516.py:1: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/insurance.txt", delimiter="|")


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,LossRatio,ClaimOccurred
0,145249,12827,2015-03-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0,False
1,145249,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0,False
2,145249,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,NaN,False
3,145255,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,0.0,False
4,145255,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,NaN,False


📊 Hypotheses Tested
1. Province vs Risk (ANOVA)

In [5]:
province_groups = df[['Province', 'LossRatio']].dropna().groupby('Province')['LossRatio'].apply(list)
f_stat, p_val = stats.f_oneway(*province_groups)


2. Zip Code vs Risk

In [6]:
top_zipcodes = df['PostalCode'].value_counts().head(2).index
group1 = df[df['PostalCode'] == top_zipcodes[0]]['LossRatio'].dropna()
group2 = df[df['PostalCode'] == top_zipcodes[1]]['LossRatio'].dropna()
t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=False)


3. Margin Difference by Zip Code

In [7]:
df['Margin'] = df['TotalPremium'] - df['TotalClaims']
group1 = df[df['PostalCode'] == top_zipcodes[0]]['Margin'].dropna()
group2 = df[df['PostalCode'] == top_zipcodes[1]]['Margin'].dropna()
t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=False)


4. Gender vs Risk

In [8]:
group_m = df[df['Gender'] == 'Male']['LossRatio'].dropna()
group_f = df[df['Gender'] == 'Female']['LossRatio'].dropna()
t_stat, p_val = stats.ttest_ind(group_m, group_f, equal_var=False)
